In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('academic_data.csv')

test = pd.read_csv('test.csv')

sample= pd.read_csv('sample_submission.csv')

train.head()

train.drop(columns=['Course'],inplace  =True)

# test.drop(columns=['Course'],inplace = True)

train.isnull().sum()

train.shape

test.shape

test.isnull().sum()

test.head()

test.drop(columns=['id'],inplace = True)

test.shape

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x = train.drop(columns=['Target'],axis = 1)

y =train['Target']

x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=42)

train.shape

test.shape

train.head()

test.drop(columns=['Course'],inplace = True)



from sklearn.preprocessig import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(test)


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [4]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), x.columns)
    ])

# Define the parameter grids for each classifier
param_grid_catboost = {
    'classifier__iterations': [100, 200, 300, 500],
    'classifier__learning_rate': [0.01, 0.03, 0.05, 0.1],
    'classifier__depth': [4, 6, 8, 10],
    'classifier__l2_leaf_reg': [1, 3, 5, 7, 9],
    'classifier__border_count': [32, 64, 128],
    'classifier__bagging_temperature': [0, 0.5, 1],
    'classifier__random_strength': [1, 2, 5, 10]
}

param_grid_xgb = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__max_depth': [4, 6, 8, 10],
    'classifier__colsample_bytree': [0.3, 0.7, 1.0],
    'classifier__subsample': [0.5, 0.7, 1.0]
}

param_grid_lgbm = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__num_leaves': [31, 63, 127],
    'classifier__min_child_samples': [10, 20, 30],
    'classifier__subsample': [0.5, 0.7, 1.0],
    'classifier__colsample_bytree': [0.5, 0.7, 1.0]
}

# Create pipelines for each classifier
pipeline_catboost = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', CatBoostClassifier(verbose=0))
])

pipeline_xgb = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
])

pipeline_lgbm = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LGBMClassifier())
])

# Set up GridSearchCV for each classifier
grid_search_catboost = GridSearchCV(estimator=pipeline_catboost, param_grid=param_grid_catboost, cv=3, scoring='accuracy')
grid_search_xgb = GridSearchCV(estimator=pipeline_xgb, param_grid=param_grid_xgb, cv=3, scoring='accuracy')
grid_search_lgbm = GridSearchCV(estimator=pipeline_lgbm, param_grid=param_grid_lgbm, cv=3, scoring='accuracy')

# Fit the models
grid_search_catboost.fit(x_train, y_train)
grid_search_xgb.fit(x_train, y_train)
grid_search_lgbm.fit(x_train, y_train)

# Get the best models
best_catboost = grid_search_catboost.best_estimator_
best_xgb = grid_search_xgb.best_estimator_
best_lgbm = grid_search_lgbm.best_estimator_

# Predict on the validation set using the best models
y_pred_catboost = best_catboost.predict(x_val)
y_pred_xgb = best_xgb.predict(x_val)
y_pred_lgbm = best_lgbm.predict(x_val)

# Evaluate the models
accuracy_catboost = accuracy_score(y_val, y_pred_catboost)
accuracy_xgb = accuracy_score(y_val, y_pred_xgb)
accuracy_lgbm = accuracy_score(y_val, y_pred_lgbm)

print(f'Best CatBoost Accuracy: {accuracy_catboost * 100:.2f}%')
print(f'Best XGBoost Accuracy: {accuracy_xgb * 100:.2f}%')
print(f'Best LightGBM Accuracy: {accuracy_lgbm * 100:.2f}%')

ValueError: 
All the 34560 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
34560 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ayush\anaconda3\Lib\site-packages\sklearn\utils\__init__.py", line 424, in _get_column_indices
    all_columns = X.columns
                  ^^^^^^^^^
AttributeError: 'numpy.ndarray' object has no attribute 'columns'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ayush\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ayush\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_step